In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def clean_csv(df_info):
    df_info["Direction"] = df_info["Direction"].str.replace("\n", " ")
    df_info["Description"] = df_info["Description"].str.replace("\n", " ")
    df_info["Room"] = df_info["Room"].str.replace("hab.", "")
    if df_info["Price"].dtype == object:
        df_info["Price"] = df_info["Price"].str.replace("€/mes", "")
    df_info["m2"] = df_info["m2"].str.replace("m²", "")
    df_info.drop(columns=["Unnamed: 0"], axis=1, inplace=True)


    return df_info

In [3]:
def changes_csv(df_info):
    # Ponemos 0 habitaciones para los estudios
    mask = df_info[df_info["Room"].str.contains("m")]["Room"].index 
    df_info.loc[mask,"Room"] = 0

    # Cambiamos a valores numericos
    df_info["m2"] = pd.to_numeric(df_info["m2"], errors="coerce")
    df_info["Room"] = pd.to_numeric(df_info["Room"], errors="coerce")
    df_info["Price"] = pd.to_numeric(df_info["Price"], errors="coerce")

    # Changed same indc
    mask_index_min = df_info[df_info["Price"]<20.000].index
    df_info.loc[mask_index_min, "Price"] = df_info.loc[mask_index_min, "Price"]*1000


    # clear rooms
    df_info["Floor"] = df_info["Floor"].str.extract(r"(\d+)")
    df_info["Floor"].fillna(0, inplace=True)


    return df_info


In [4]:
df = pd.read_csv("../data/rentaMadrid2023_7.csv") 
df

,Unnamed: 0,Direction,Price,m2,Floor,Room,Description
0,0,"\nPiso en calle de los Fundadores, 23, Fuente ...",1.200€/mes,65 m²,Planta 3ª interior con ascensor,2 hab.,\nCALEFACCIÓN CENTRAL Y PLAZA DE GARAJE INCLUI...
1,1,"\nPiso en paseo de San Francisco de Sales, Val...",3.500€/mes,200 m²,Planta 2ª exterior con ascensor,4 hab.,\nCONTRATO DE TEMPORADA: MÁXIMO 11 MESES. WIFI...
2,2,"\nPiso en Canillas, Prosperidad, Madrid\n",1.545€/mes,30 m²,Entreplanta exterior con ascensor,1 hab.,\nESPECTACULAR APARTAMENTO EN PROSPERIDAD!\nAl...
3,3,"\nPiso en calle de Serrano, El Viso, Madrid\n",2.600€/mes,120 m²,Bajo exterior con ascensor,3 hab.,\nAlquiler de magnifico piso con amplio jardín...
4,4,"\nPiso en calle del Cardenal Cisneros, 82, Tra...",1.300€/mes,50 m²,Planta -1 interior sin ascensor,2 hab.,"\nPrecioso apartamento de dos habitaciones, si..."
5,5,"\nEstudio en calle de Meléndez Valdés, Gaztamb...",900€/mes,Planta 2ª interior con ascensor,NaN,40 m²,\nALQUILOFACIL MONCLOA alquila espectacular es...
6,6,"\nDúplex en Butron, Rosas, Madrid\n",1.150€/mes,77 m²,Planta 2ª exterior con ascensor,2 hab.,\nSe alquila vivienda con casi 5 metros de alt...
7,7,"\nPiso en calle de Alonso Cano, Nuevos Ministe...",1.600€/mes,73 m²,Planta 2ª interior con ascensor,1 hab.,"\nSe alquila en Chamberí, piso reformado para ..."
8,8,"\nPiso en carabanchel, 2, Puerta Bonita, Madrid\n",790€/mes,65 m²,Planta 4ª exterior sin ascensor,3 hab.,"\nVivienda 58 m, totalmente reformada, Calefac..."
9,9,"\nChalet adosado en calle de Rafael Villa, El ...",2.950€/mes,270 m²,NaN,4 hab.,\nChalet reformado adosado con anexo para serv...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   30 non-null     int64 
 1   Direction    30 non-null     object
 2   Price        30 non-null     object
 3   m2           30 non-null     object
 4   Floor        27 non-null     object
 5   Room         30 non-null     object
 6   Description  30 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


In [6]:
df = clean_csv(df)
df = changes_csv(df)
df

,Direction,Price,m2,Floor,Room,Description
0,"Piso en calle de los Fundadores, 23, Fuente d...",1200.0,65.0,3,2,CALEFACCIÓN CENTRAL Y PLAZA DE GARAJE INCLUID...
1,"Piso en paseo de San Francisco de Sales, Vall...",3500.0,200.0,2,4,CONTRATO DE TEMPORADA: MÁXIMO 11 MESES. WIFI ...
2,"Piso en Canillas, Prosperidad, Madrid",1545.0,30.0,0,1,ESPECTACULAR APARTAMENTO EN PROSPERIDAD! Alqu...
3,"Piso en calle de Serrano, El Viso, Madrid",2600.0,120.0,0,3,Alquiler de magnifico piso con amplio jardín....
4,"Piso en calle del Cardenal Cisneros, 82, Traf...",1300.0,50.0,1,2,"Precioso apartamento de dos habitaciones, sit..."
5,"Estudio en calle de Meléndez Valdés, Gaztambi...",900.0,NaN,0,0,ALQUILOFACIL MONCLOA alquila espectacular est...
6,"Dúplex en Butron, Rosas, Madrid",1150.0,77.0,2,2,Se alquila vivienda con casi 5 metros de altu...
7,"Piso en calle de Alonso Cano, Nuevos Minister...",1600.0,73.0,2,1,"Se alquila en Chamberí, piso reformado para p..."
8,"Piso en carabanchel, 2, Puerta Bonita, Madrid",790.0,65.0,4,3,"Vivienda 58 m, totalmente reformada, Calefacc..."
9,"Chalet adosado en calle de Rafael Villa, El P...",2950.0,270.0,0,4,Chalet reformado adosado con anexo para servi...


In [10]:
# Concat principal csv
df_idealista = pd.read_csv("../data/rentaMadrid2023Idealista.csv", sep =";")
# Clear data
df_idealista

,Direction,Price,m2,Floor,Room,Description
0,"Piso en calle de Galileo, Arapiles, Madrid",3300.0,55.0,6,1,BeHomeMadrid ofrece coqueto apartamento A EST...
1,"Piso en calle del Pintor Ignacio de Zuloaga, ...",1345.0,84.0,0,2,¡Sin comisiones de agenia! Y AHORA ALQUILAN...
2,"Piso en Pau de Carabanchel, Madrid",1500.0,102.0,4,3,Inmohappy Madrid Norte Les Ofrece esta Amplia...
3,"Piso en calle Prieto Ureña, Nueva España, Mad...",5500.0,297.0,1,5,INMOBILIARIA EMMANUEL PARA ENTRAR 1 de DICIEM...
4,"Piso en plaza Martinez Olmedilla, Santa Eugen...",1050.0,120.0,3,4,"Vivienda reformada de 120 m2, 4 amplios dormi..."
...,...,...,...,...,...,...
175,"Piso en Castellana, Madrid",4200.0,230.0,5,2,La Inmobiliaria Internacional CPM gestiona es...
176,"Piso en German Pérez Carrasco, Quintana, Madrid",900.0,50.0,2,2,"¿Estás buscando una vivienda luminosa, recién..."
177,"Piso en calle de Hermosilla, 18, Recoletos, M...",3900.0,170.0,2,3,MAGNÍFICA VIVIENDA EN PLENO BARRIO DE SALAMAN...
178,"Estudio en calle del Manco de Lepanto, Numanc...",630.0,NaN,3,0,POR FAVOR LEER TODA LA DESCRIPCIÓN DEL ANUNCI...


In [11]:
df_final = pd.concat([df,df_idealista], ignore_index=True)

In [12]:
df_final.to_csv("../data/rentaMadrid2023Idealista.csv", sep=";", index=False)

In [13]:
df_final

,Direction,Price,m2,Floor,Room,Description
0,"Piso en calle de los Fundadores, 23, Fuente d...",1200.0,65.0,3,2,CALEFACCIÓN CENTRAL Y PLAZA DE GARAJE INCLUID...
1,"Piso en paseo de San Francisco de Sales, Vall...",3500.0,200.0,2,4,CONTRATO DE TEMPORADA: MÁXIMO 11 MESES. WIFI ...
2,"Piso en Canillas, Prosperidad, Madrid",1545.0,30.0,0,1,ESPECTACULAR APARTAMENTO EN PROSPERIDAD! Alqu...
3,"Piso en calle de Serrano, El Viso, Madrid",2600.0,120.0,0,3,Alquiler de magnifico piso con amplio jardín....
4,"Piso en calle del Cardenal Cisneros, 82, Traf...",1300.0,50.0,1,2,"Precioso apartamento de dos habitaciones, sit..."
...,...,...,...,...,...,...
205,"Piso en Castellana, Madrid",4200.0,230.0,5,2,La Inmobiliaria Internacional CPM gestiona es...
206,"Piso en German Pérez Carrasco, Quintana, Madrid",900.0,50.0,2,2,"¿Estás buscando una vivienda luminosa, recién..."
207,"Piso en calle de Hermosilla, 18, Recoletos, M...",3900.0,170.0,2,3,MAGNÍFICA VIVIENDA EN PLENO BARRIO DE SALAMAN...
208,"Estudio en calle del Manco de Lepanto, Numanc...",630.0,NaN,3,0,POR FAVOR LEER TODA LA DESCRIPCIÓN DEL ANUNCI...
